# Play by Play Gatherer

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from urllib.request import urlopen

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
requestHead = {"User-Agent": "Chrome/47.0.2526.111"}

## General

In [3]:
def getSeasonIDs(num,online = True,prefix = None):
    if online:
        if num < 10:
            strnum = '0' + str(num)
        elif num >= 10:
            strnum = str(num)
        url = "http://sim-football.com/indexes/NSFLS"+strnum+"/GameResults.html"
        page = requests.get(url, headers = requestHead)
        soup = BeautifulSoup(page.text,'html.parser')
    else:
        with open(prefix+'/GameResults.html') as f:
            soup = BeautifulSoup(f,'html.parser')
    
    
    pbplist = soup.find_all('a',href=re.compile('Logs'))
    pbpURLs = [p.get('href') for p in pbplist]
    idList = [p[5:].strip('.html') for p in pbpURLs]
    return idList

In [4]:
# def teams(teamID):
#     teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'NO ','9':'CHI','10':'AUS'}
#     return teamIDs[teamID]

In [5]:
def dist2goal(team,side,yard):
    if team == side:
        dist = 100 - int(yard)
    elif side != '':
        dist = int(yard)
    else:
        dist = ''
    return dist

In [6]:
def goal2go(distance,dist2goal):
    if distance == 'Goal':
        return int(dist2goal)
    elif distance == '':
        return ''
    elif distance == 'inches':
        return 1.
    else:
        return int(distance)

In [7]:
def getScore(boxDF,totTime):
    i = 0
    if int(totTime) > 0:
#     print('\n')
#     print(totTime)
        while int(totTime) <= int(boxDF.loc[i+1]['totTime']):
            i += 1
        return boxDF.loc[i]['awayScore'],boxDF.loc[i]['homeScore']
    else:
        return boxDF.loc[len(boxDF)-1]['awayScore'],boxDF.loc[len(boxDF)-1]['homeScore']

In [8]:
def getGameData(S,gameID,online=True,prefix = None):
    
    def getteams(teamID):
        if S < 5:
            teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'ARI','5':'OCO','6':'SJS','7':'PHI','8':'LV ','9':'CHI','10':'AUS'}
        elif S == 5:
            teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'LV ','9':'CHI','10':'AUS'}
        else:
            teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'NO ','9':'CHI','10':'AUS'}
        return teamIDs[teamID]
    if online == True:
        if S < 10:
            strnum = '0' + str(S)
        elif S >= 10:
            strnum = str(S)
        pagePBP = requests.get("http://sim-football.com/indexes/NSFLS"+strnum+"/Logs/"+gameID+".html", headers = requestHead)
        pageBox = requests.get("http://sim-football.com/indexes/NSFLS"+strnum+"/Boxscores/"+gameID+".html", headers = requestHead)
        soupPBP = BeautifulSoup(pagePBP.content,'lxml')
        soupBox = BeautifulSoup(pageBox.content,'lxml')
    else:
        with open(prefix+'/Logs/'+gameID+'.html') as pagePBP:
            soupPBP = BeautifulSoup(pagePBP,'lxml')
        with open(prefix+'/Boxscores/'+gameID+'.html') as pageBox:
            soupBox = BeautifulSoup(pageBox,'lxml')
    
    tableBox = soupBox.find_all('table', class_='Grid')[0]
    tablePBP = soupPBP.find_all('table',class_='Grid')[0]
    
    q = -2
    pbpList = []
    for row in tablePBP.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) != 5:
            q = q + 1
        elif len(cols) == 5:
            team = cols[0].find_all('img')[0]['src'][16:].strip('_s.png')
            textTime = cols[1].text
            minutes,seconds = cols[1].text.strip().split(':')
            secondsLeft = int(minutes)*60 + int(seconds)
            tottime = 15*60*(4-q) + secondsLeft
            downDist = cols[2].text
            if downDist == '':
                down = ''
                dist = ''
            elif downDist == '---':
                down = ''
                dist = ''
            else:
                down = int(downDist[0])
                dist = downDist[8:]
            loc = cols[3].text
            if loc == '':
                side = yard = ''
            else:
                side = loc[:3]
                yard = int(loc[-2:])
            play = cols[4].text
            pbpList.append((team,q,textTime,tottime,down,dist,side,yard,play))
            
    pbpArr = np.asarray(pbpList)
    pbpDF = pd.DataFrame(pbpArr)
    pbpDF.columns = ['teamID','Q','time','totTime','down','distance','side','yard','play']
    pbpDF['gameID'] = gameID
    pbpDF['S'] = S
    
    teamList = list(pbpDF['side'].unique())
    teamList.remove('')
    homeTeam = teamList[0]
    awayTeam = teamList[-1]
    pbpDF['homeTeam'] = homeTeam
    pbpDF['awayTeam'] = awayTeam
    
    pbpDF['teamPoss'] = pbpDF.apply(lambda row : getteams(row['teamID']),axis=1)
    pbpDF['dist2goal'] = pbpDF.apply(lambda row : dist2goal(row['teamPoss'],row['side'],row['yard']),axis=1)
    pbpDF['distance'] = pbpDF.apply(lambda row : goal2go(row['distance'],row['dist2goal']),axis=1)
    
    
    
    scoreList = [(3600,0,0)]
    
    q=0
    for row in tableBox.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) != 6:
            q += 1
        else:
            time = cols[-4].text.split(':')
            secondsLeft = int(time[0]) * 60 + int(time[1])
            totTime = 15*60*(4-q) + secondsLeft
            awayscore = int(cols[-2].text)
            homescore = int(cols[-1].text)
            scoreList.append((totTime,awayscore,homescore))
    scoreList.append((0,awayscore,homescore))
        
    boxArr = np.asarray(scoreList)
    boxDF = pd.DataFrame(boxArr)
    boxDF.columns = ['totTime','awayScore','homeScore']
    
    pbpDF['awayScore'] = pbpDF.apply(lambda row : getScore(boxDF,row['totTime'])[0],axis=1)
    pbpDF['homeScore'] = pbpDF.apply(lambda row : getScore(boxDF,row['totTime'])[1],axis=1)
    
#     pbpDF = pbpDF[['S','gameID','homeTeam','awayTeam','teamPoss','Q','time','totTime','awayScore','homeScore','down','distance','dist2goal','play']]
    
    return pbpDF

In [9]:
s3Data = []
idList = getSeasonIDs(6)
for i in idList:
    s3Data.append(getGameData(6,i))

In [10]:
s3DF = pd.concat(s3Data)
s3DF.to_csv('s6.csv')

In [11]:
allData = []

In [12]:
indexList = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

In [13]:
sBar = tqdm(total=len(indexList),desc='S')
for s in indexList:
    sBar.set_description('S'+str(s))
    idList = getSeasonIDs(s)
    print(len(idList))
    idBar = tqdm(total=len(idList),desc='id')
    for i in idList:
        idBar.set_description(i)
        allData.append(getGameData(s,i))
        idBar.update(1)
    sBar.update(1)

57


74


75


75


75


...

75


75


92


92


92


In [14]:
# offlineIndexList = [7,8,13]
# offlinePrefixList = ['NSFL S7PW2 Complete/HTML','NSFL S8PW2 Complete/HTML','S13Ultimus/HTML']

In [15]:
# s2bar = tqdm(total=len(offlineIndexList),desc='S')
# for s in range(len(offlineIndexList)):
#     s2bar.set_description('S'+str(offlineIndexList[s]))
#     idList = getSeasonIDs(offlineIndexList[s],online=False,prefix=offlinePrefixList[s])
#     idbar = tqdm(total=len(idList),desc='id')
#     for i in idList:
#         idbar.set_description(i)
#         allData.append(getGameData(offlineIndexList[s],i,online=False,prefix=offlinePrefixList[s]))

In [16]:
allDF = pd.concat(allData)

In [17]:
allDF.to_csv('all.csv')

In [1]:
allDF

NameError: name 'allDF' is not defined

## Testing

In [290]:
strnum = '05'

url = "http://sim-football.com/indexes/NSFLS"+strnum+"/GameResults.html"

page = requests.get(url,  headers = {"User-Agent": "Chrome/47.0.2526.111"})
soup = BeautifulSoup(page.text,'html.parser')

In [304]:
PbPList = soup.find_all('a', href = re.compile("Logs"))
PbPList = [p.get('href') for p in PbPList]
PbPList[0]

'Logs/1358.html'

In [305]:
idList = [p[5:].strip('.html') for p in PbPList]

In [306]:
idList[0]

'1358'

In [307]:
page2 = requests.get("http://sim-football.com/indexes/NSFLS"+strnum+"/Logs/"+idList[0]+".html", headers = requestHead)

In [308]:
soup3 = BeautifulSoup(page2.content,'lxml')

In [309]:
table = soup3.find_all('table', class_='Grid')[0]

In [310]:
q = -2
pbpList = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
#     print(cols)
    if len(cols) != 5:
        q = q + 1
    elif len(cols) == 5:
        team = cols[0].find_all('img')[0]['src'][16:].strip('_s.png')
        textTime = cols[1].text
        minutes,seconds = cols[1].text.strip().split(':')
        secondsLeft = int(minutes)*60 + int(seconds)
        tottime = 15*60*(4-q) + secondsLeft
        downDist = cols[2].text
        if downDist == '':
            down = ''
            dist = ''
        elif downDist == '---':
            down = ''
            dist = ''
        else:
            down = int(downDist[0])
            dist = downDist[8:]
#         print(cols[3])
        loc = cols[3].text
        if loc == '':
            side = yard = ''
        else:
            side = loc[:3]
            yard = int(loc[-2:])
        play = cols[4].text
        pbpList.append((team,q,textTime,tottime,down,dist,side,yard,play))

In [311]:
pbpArr = np.asarray(pbpList)
df = pd.DataFrame(pbpArr)
df.columns = ['teamID','Q','time','totTime','down','distance','side','yard','play']

In [312]:
teamList = list(df['side'].unique())
teamList.remove('')
homeTeam = teamList[0]
awayTeam = teamList[-1]

In [313]:
teamList

['PHI', 'SJS']

In [314]:
teamList

['PHI', 'SJS']

In [315]:
df['teamPoss'] = df.apply(lambda row : teams(row['teamID']),axis=1)

KeyError: ('', 'occurred at index 0')

In [259]:
df['dist2goal'] = df.apply(lambda row : dist2goal(row['teamPoss'],row['side'],row['yard']),axis=1)

In [260]:
df['distance'] = df.apply(lambda row : goal2go(row['distance'],row['dist2goal']),axis=1)

In [261]:
df = df[['teamPoss','Q','time','totTime','down','distance','dist2goal','play']]

In [262]:
df

,teamPoss,Q,time,totTime,down,distance,dist2goal,play
0,ARI,1,15:00,3600,,,30,"Jackson, D. kicks off."
1,NO,1,15:00,3600,,,88,"Kickoff of 62 yards. Returned by Smallwood, D...."
2,NO,1,14:57,3597,1,10,88,"Pass by Maximus III, B., complete to DiMirio, ..."
3,NO,1,14:18,3558,1,10,73,"Rush by Smallwood, D. for 5 yds. Tackle by Ern..."
4,NO,1,13:59,3539,2,5,68,"Pass by Maximus III, B. to Law, C. is incomple..."
5,NO,1,13:55,3535,3,5,68,"Rush by Smallwood, D. for 4 yds. Tackle by Ern..."
6,ARI,1,13:30,3510,4,1,36,"Punt by Blewitt, I. of 34 yards."
7,ARI,1,13:30,3510,,,,"Returned by Morris, D. for -3 yards."
8,ARI,1,13:17,3497,1,10,27,"Pass by Fitzpatrick, K. to Mackworthy, R. was ..."
9,ARI,1,13:13,3493,2,10,27,"Pass by Fitzpatrick, K., complete to Squanch, ..."


***

## Boxscore

In [263]:
boxList = soup.find_all('a', href = re.compile("Boxscore"))
boxList = [p.get('href') for p in boxList]
boxList[4]

'Boxscores/2305.html'

In [264]:
pagebox = requests.get("http://sim-football.com/indexes/NSFLS"+strnum+"/"+boxList[4], headers = requestHead)

In [265]:
soupbox = BeautifulSoup(pagebox.content,'lxml')

In [266]:
tablebox = soupbox.find_all('table', class_='Grid')[0]

In [267]:
scoreList = [(3600,0,0)]
# scoreList = []

In [268]:
q=0
for row in tablebox.find_all('tr')[1:]:
    cols = row.find_all('td')
#     print(len(cols))
    if len(cols) != 6:
        q += 1
#         scoreList.append((q,901,awayscore,homescore))
    else:
#         print(cols[-4])
        time = cols[-4].text.split(':')
        secondsLeft = int(time[0]) * 60 + int(time[1])
        totTime = 15*60*(4-q) + secondsLeft
        awayscore = int(cols[-2].text)
        homescore = int(cols[-1].text)
        scoreList.append((totTime,awayscore,homescore))
scoreList.append((0,awayscore,homescore))

In [269]:
scoreList

[(3600, 0, 0),
 (3003, 3, 0),
 (2555, 10, 0),
 (2060, 17, 0),
 (1417, 24, 0),
 (1222, 31, 0),
 (877, 31, 2),
 (701, 34, 2),
 (494, 34, 9),
 (287, 41, 9),
 (73, 44, 9),
 (0, 44, 9)]

In [270]:
boxArr = np.asarray(scoreList)
boxdf = pd.DataFrame(boxArr)
boxdf.columns = ['totTime','awayScore','homeScore']

In [271]:
boxdf

,totTime,awayScore,homeScore
0,3600,0,0
1,3003,3,0
2,2555,10,0
3,2060,17,0
4,1417,24,0
5,1222,31,0
6,877,31,2
7,701,34,2
8,494,34,9
9,287,41,9


In [272]:
df['awayScore'] = df.apply(lambda row : getScore(boxdf,row['totTime'])[0],axis=1)

In [273]:
df['homeScore'] = df.apply(lambda row : getScore(boxdf,row['totTime'])[1],axis=1)

In [274]:
teamList[-1]

'NO '

In [275]:
df['homeTeam'] = homeTeam
df['awayTeam'] = awayTeam
df['gameID'] = PbPList[0][5:9]

In [276]:
teamList

['AZ ', 'NO ']

In [277]:
df = df[['gameID','homeTeam','awayTeam','teamPoss','Q','time','totTime','awayScore','homeScore','down','distance','dist2goal','play']]

In [278]:
len(df)

213

In [279]:
df.loc[50:100]

,gameID,homeTeam,awayTeam,teamPoss,Q,time,totTime,awayScore,homeScore,down,distance,dist2goal,play
50,2301,AZ,NO,NO,2,13:45,2625,3,0,3,3,18,"Pass by Maximus III, B., complete to Smallwood..."
51,2301,AZ,NO,NO,2,13:06,2586,3,0,1,10,13,"Pass by Maximus III, B. to Law, C. falls incom..."
52,2301,AZ,NO,NO,2,13:03,2583,3,0,2,10,13,"Pass by Maximus III, B., complete to Law, C. f..."
53,2301,AZ,NO,NO,2,12:35,2555,10,0,,,,TOUCHDOWN! (Blewitt kick good)
54,2301,AZ,NO,NO,2,12:35,2555,10,0,,,70,"Blewitt, I. kicks off."
55,2301,AZ,NO,ARI,2,12:35,2555,10,0,,,24,"Kickoff of 63 yards. Returned by Mackworthy, R..."
56,2301,AZ,NO,ARI,2,12:32,2552,10,0,1,10,24,"Pass by Fitzpatrick, K. to Chess, C. was dropp..."
57,2301,AZ,NO,ARI,2,12:29,2549,10,0,2,10,24,"Pass by Fitzpatrick, K., complete to Chess, C...."
58,2301,AZ,NO,ARI,2,11:49,2509,10,0,3,5,29,"Rush by Mackworthy, R. for 1 yds. Tackle by As..."
59,2301,AZ,NO,NO,2,11:28,2488,10,0,4,4,70,"Punt by Jackson, D. of 33 yards."


### punts

In [148]:
fourthDF = df[df['down'].str.contains('4')]

In [149]:
def playType(play):
#     print(play)
    if 'Punt' in play:
        ptype = 'Punt'
    elif 'FG' in play:
        ptype = 'FG'
    else:
        ptype = 'Try'
    return ptype

In [150]:
fourthDF['playType'] = fourthDF.apply(lambda row : playType(row['play']),axis=1)

/Users/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [151]:
fourthDF['playType'].value_counts()

Punt    15
Try      3
FG       1
Name: playType, dtype: int64

In [152]:
fgDF = df[df['play'].str.contains('FG')]

In [153]:
len(fgDF)

2

In [154]:
puntDF = df[df['play'].str.contains('Punt')]
puntDF = puntDF.rename(columns = {'teamPoss':'recTeam'})

In [155]:
len(puntDF)

15

In [156]:
puntDF['surrenderIndex'] = puntDF.apply(lambda row : surrenderIndex(row),axis=1)


In [157]:
puntDF

,gameID,homeTeam,awayTeam,recTeam,Q,time,totTime,awayScore,homeScore,down,distance,dist2goal,play,surrenderIndex
7,4203,YKW,BAL,YKW,1,14:27,3567,0,0,4,10,53,"Punt by Brand, F. of 53 yards. Touchback.",0.115866
20,4203,YKW,BAL,YKW,1,9:08,3248,0,7,4,7,66,"Punt by Brand, F. of 52 yards. No return.",1.600000
46,4203,YKW,BAL,BAL,1,1:02,2762,7,14,4,4,79,"Punt by Bobby (R), K. of 46 yards.",0.600000
83,4203,YKW,BAL,BAL,2,8:04,2284,28,14,4,7,77,"Punt by Bobby (R), K. of 47 yards.",1.200000
91,4203,YKW,BAL,BAL,2,6:09,2169,28,14,4,16,44,"Punt by Bobby (R), K. of 44 yards. Touchback.",4.646924
95,4203,YKW,BAL,YKW,2,5:28,2128,28,14,4,9,79,"Punt by Brand, F. of 30 yards. No return.",0.400000
124,4203,YKW,BAL,BAL,3,13:50,1730,28,21,4,13,75,"Punt by Bobby (R), K. of 41 yards. No return.",0.800274
131,4203,YKW,BAL,YKW,3,11:54,1614,28,21,4,15,56,"Punt by Brand, F. of 32 yards.",0.043526
152,4203,YKW,BAL,BAL,3,4:46,1186,35,21,4,2,60,"Punt by Bobby (R), K. of 46 yards. No return.",2.955541
159,4203,YKW,BAL,YKW,3,2:49,1069,35,21,4,14,80,"Punt by Brand, F. of 46 yards. No return.",0.200000
